In [102]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

import torchvision
from torchvision import datasets, models, transforms

from tqdm.notebook import tqdm

In [103]:
resnet_transform  = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomPerspective(distortion_scale = 0.6, p = 1.0),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [104]:
full_data = datasets.ImageFolder('./emotion/test',transform = resnet_transform)


train_size = int(len(full_data)*0.8)
val_size = int(len(full_data)*0.1)
test_size = len(full_data) - train_size - val_size

train_data , val_test_data = torch.utils.data.random_split(full_data, [train_size, val_size+test_size])
val_data, test_data = torch.utils.data.random_split(val_test_data, [val_size, test_size])

In [105]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size = BATCH, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size = BATCH, shuffle = False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = BATCH, shuffle = False)

In [106]:
BATCH = 64
answer = ["angry", "disgust","fear","happy","neutral","sad", "suprised"]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [107]:
for i in train_loader:
    print(i)
    break

[tensor([[[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044, -1.8044, 

In [108]:
model = models.resnet18(weights = True)

In [109]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [110]:
model.fc = nn.Linear(512,7)

In [111]:
def evaluate(model, dataloader, loss_fn):
    losses = []
    
    num_correct =0
    num_elements = 0
    for i,batch in tqdm(enumerate(dataloader)):
        X_batch, y_batch = batch
        X_batch = X_batch.to(DEVICE)
        y_batch = y_batch.to(DEVICE)
        num_elements+=len(y_batch)
        
        with torch.no_grad():
            logits = model(X_batch.to(DEVICE))
            
            loss = loss_fn(logits,y_batch.to(DEVICE))
            losses.append(loss.item())
            
            y_pred  = torch.argmax(logits, dim=1)
            
            num_correct+=torch.sum(y_pred==y_batch).cpu()
    accuracy = num_correct/num_elements
    return accuracy.numpy(), np.mean(losses)

In [112]:
def train(model, loss_fn, optimazer, n_epoch= 3):
    for epoch in tqdm(range(n_epoch)):
        print("epoch #", epoch)
        losses=[]
        running_acc=[]
        model.train(True)
        for i,batch in tqdm(enumerate(train_loader)):            
            X_batch, y_batch = batch
            X_batch = X_batch.to(DEVICE)
            y_batch = y_batch.to(DEVICE)
            
            logits = model(X_batch)
            
            loss = loss_fn(logits,y_batch)
            losses.append(loss.item())
            
            loss.backward()
            optimazer.step()
            optimazer.zero_grad()
            
            model_ans = torch.argmax(logits, dim=1)
            train_acc = torch.sum(y_batch == model_ans)/ len(y_batch)
            running_acc.append(train_acc.cpu())
            
            if(i+1)%40 ==0:
                print("Средние train лосс и accuracy на последних 40 итерациях:",
                      np.mean(losses), np.mean(running_acc), end='\n')
                
        model.train(False)

        val_acc, val_loss = evaluate(model, val_loader, loss_fn)
        print("Эпоха {}/{}: val лосс и accuracy:".format(epoch+1, n_epoch,),
                  val_loss, val_acc, end='\n')
    return model
            

In [113]:
model.to(DEVICE)
loss_fn = nn.CrossEntropyLoss()
lr = 1e-3
optim = torch.optim.Adam(model.parameters(), lr = lr)

In [118]:
model = train(model, loss_fn, optim, 15)

  0%|          | 0/15 [00:00<?, ?it/s]

epoch # 0


0it [00:00, ?it/s]

Средние train лосс и accuracy на последних 40 итерациях: 0.8066565439105033 0.7019531
Средние train лосс и accuracy на последних 40 итерациях: 0.8196961477398872 0.6904297


0it [00:00, ?it/s]

...

Эпоха 14/15: val лосс и accuracy: 1.7092249691486359 0.54
epoch # 14


0it [00:00, ?it/s]

Средние train лосс и accuracy на последних 40 итерациях: 0.403250328451395 0.8539063
Средние train лосс и accuracy на последних 40 итерациях: 0.42790679857134817 0.85


0it [00:00, ?it/s]

Эпоха 15/15: val лосс и accuracy: 1.8336058557033539 0.548


In [119]:
evaluate(model,test_loader, loss_fn)

0it [00:00, ?it/s]

(array(0.5425532, dtype=float32), 1.6629248559474945)

In [127]:
model.cpu()
torch.save(model, "extra_model1.pth")

In [67]:
model = torch.load("model.pth")
model.eval()

RecursiveScriptModule(
  original_name=ResNet
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
  (relu): RecursiveScriptModule(original_name=ReLU)
  (maxpool): RecursiveScriptModule(original_name=MaxPool2d)
  (layer1): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=BasicBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (conv2): RecursiveScriptModule(original_name=Conv2d)
      (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
    )
    (1): RecursiveScriptModule(
      original_name=BasicBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (conv2): RecursiveScriptModule(original_name=Conv2d)

In [126]:
from PIL import Image
img = Image.open("./images/7/Disgust.jpg")
img = resnet_transform(img)
model.cpu()
answer[np.argmax(model(img.reshape(1,3,224,224)).detach())]
#model(img.reshape(1,3,224,224))

'suprised'